In [1]:
from Crypto.Hash import HMAC
from Crypto.Hash import SHA as SHA1
from Crypto.Util.strxor import strxor
from binascii import hexlify, unhexlify

def hmacsha1(key, message):
    return HMAC.new(key, message, SHA1).hexdigest() # Note: output is revealed in hex



def leaky_hmac_verify(key, message, claimed_tag):
    # Assume that the tag is well-formed, so it's even possible to be the hex encoding of an HMAC-SHA1 output (which would be 20 bytes long)
    assert(len(claimed_tag) == 40)

    # Test validity of the claimed tag
    valid_tag = hmacsha1(key, message)                         # This is what the tag should be, in hex

    is_valid_tag = (claimed_tag == valid_tag)

    if(is_valid_tag):                                          # The tag is valid, so the "first difference" is after the end of the string
        return [is_valid_tag, 4 * len(valid_tag)]
    else:                                                      # The tag is invalid, and we must find the location of the first difference
        diff = hexlify(strxor(unhexlify(claimed_tag),          # To do so, we take the xor between the (raw) tag and valid_tag
                              unhexlify(valid_tag)))           # and then find the first non-zero bit in this string (which is easier to do when hexlify'd)
        diffstrip = diff.lstrip("0".encode())                       # Remove all of the leading hex-0 characters
        first_diff_location = 4 * (len(diff) - len(diffstrip)) # Each leading hex-0 denotes four bits that are identical between the two strings
        
        #print("diff",diff)
        
        #print("diffstrip",diffstrip)
        
        #print("first_diff_location",first_diff_location)
        
        
        char = chr(diffstrip[0])                              # This character is guaranteed to be a non-zero hex character
        leading_bits = {'1' : 3,                               # This dictionary provides the # of leading zero bits for each non-zero hex character
                        '2' : 2,
                        '3' : 2,
                        '4' : 1,
                        '5' : 1,
                        '6' : 1,
                        '7' : 1,
                        '8' : 0,
                        '9' : 0,
                        'a' : 0,
                        'b' : 0,
                        'c' : 0,
                        'd' : 0,
                        'e' : 0,
                        'f' : 0,}
        first_diff_location += leading_bits[char]
        return [is_valid_tag, first_diff_location]             # Return whether the tag is correct *and* the location of the first difference

In [7]:
def forge(key):
    key = unhexlify(key)
    message = b"This message was definitely sent by Alice"
    claimed_tag = "0acd1f0248b85cf5f26ab4102110ae99c8de8188"
    #d19636b72f2b71d8a87d922df0108d471cbd58c3
    answer = leaky_hmac_verify(key, message, claimed_tag)
    print("first",answer[1])
    index = 0
    iterate_this = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f']
    
    max_answer = answer[1]
    
    while(answer[1]!=160):
        
        
        
        
        #print(index,answer[1]-index,(answer[1]-index) % 4)
        if((max_answer -index) % 4 == 0):
            #print("error at:",answer[1]-index)
            for i in iterate_this:
                
                if i=='0':
                    j =(int((max_answer-index)/4))
                    
                print(claimed_tag[:j])
                print(i)
                print(claimed_tag[j+1:])
                claimed_tag = claimed_tag[:j]+i+claimed_tag[j+1:]
                print(claimed_tag)
                answer = leaky_hmac_verify(key, message, claimed_tag)
                print(claimed_tag,answer[1])
                
                
                if answer[1] == 160:
                    #print("hello",claimed_tag)
                    max_i = i
                    break
                
                
                if max_answer < answer[1]:
                    max_answer = answer[1]
                    max_i = i
                    
                
                print("entering and saving as answer is ",max_answer,i,index,j, max_i)
                
            claimed_tag = claimed_tag[:j]+max_i+claimed_tag[j+1:]
            print(claimed_tag )
                    #break
                    
                    
                
                
                
                
            
            
            
        index = index+1
        
        if index == 4:
            index=0
        
        
            
        print("---------")    
            
            
            
        
    
    
    

In [8]:
forge("7477656e74792062797465206c6f6e67206b6579")

first 156
0acd1f0248b85cf5f26ab4102110ae99c8de818
0

0acd1f0248b85cf5f26ab4102110ae99c8de8180
0acd1f0248b85cf5f26ab4102110ae99c8de8180 157
entering and saving as answer is  157 0 0 39 0
0acd1f0248b85cf5f26ab4102110ae99c8de818
1

0acd1f0248b85cf5f26ab4102110ae99c8de8181
0acd1f0248b85cf5f26ab4102110ae99c8de8181 157
entering and saving as answer is  157 1 0 39 0
0acd1f0248b85cf5f26ab4102110ae99c8de818
2

0acd1f0248b85cf5f26ab4102110ae99c8de8182
0acd1f0248b85cf5f26ab4102110ae99c8de8182 157
entering and saving as answer is  157 2 0 39 0
0acd1f0248b85cf5f26ab4102110ae99c8de818
3

0acd1f0248b85cf5f26ab4102110ae99c8de8183
0acd1f0248b85cf5f26ab4102110ae99c8de8183 157
entering and saving as answer is  157 3 0 39 0
0acd1f0248b85cf5f26ab4102110ae99c8de818
4

0acd1f0248b85cf5f26ab4102110ae99c8de8184
0acd1f0248b85cf5f26ab4102110ae99c8de8184 158
entering and saving as answer is  158 4 0 39 4
0acd1f0248b85cf5f26ab4102110ae99c8de818
5

0acd1f0248b85cf5f26ab4102110ae99c8de8185
0acd1f0248b85cf5f26ab41021

In [4]:
len("d19636b72f2b71d8a87d922df0108d471cbd58c3")

40

In [5]:
key = unhexlify("7477656e74792062797465206c6f6e67206b6579")
message = unhexlify("a70c430ebf35441874ac9f758c59ee10")
claimed_tag = "d19236b72f2b71d8a87d922df0108d471cbd58c3"
    #d19636b72f2b71d8a87d922df0108d471cbd58c3
answer = leaky_hmac_verify(key, message, claimed_tag)
print(answer)

[False, 13]


In [6]:
 hmacsha1(unhexlify("7477656e74792062797465206c6f6e67206b6579"),b"This message was definitely sent by Alicd")
    

'626f083f408955433a6931ad8ac89f024d50f1fe'